In [89]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import djvu.decode

def isArmenian(s):
    return any(unicodeblock.blocks.of(c) == 'ARMENIAN' for c in s)

def isCyrillic(s):
    return any(unicodeblock.blocks.of(c) == 'CYRILLIC' for c in s)

def fixDoubleLetter(word):
    if word in ['դդում', 'թթու', 'կկու', 'շշուկ', 'խխունջ', 'ծծակ', 'թթվածին', 'թթվասեր']:
        return word
    if len(word) > 1:
        if word[0] == word[1]:
            return word[1:]
    return word

print(djvu.decode.DDJVU_VERSION)

26


In [90]:
context = djvu.decode.Context()
document = context.new_document(djvu.decode.FileUri('data/hayeren.djvu'))
document.decoding_job.wait()

In [91]:
pages = []

for page in document.pages:
    val = page.text.sexpr.value

    line_delims_and_words = [y for x in val if type(x) == tuple for y in x if type(y) != int]

    lines = []
    current_line = []

    for entry in line_delims_and_words[1:]:
        if type(entry) == tuple:
            current_line.append(entry[-1])
        else:
            lines.append(' '.join(current_line))
            current_line = []
    lines.append(' '.join(current_line))

    pages.append(lines)
    
# get section headers from pages 4-11, ignore page numbers in page headers
section_headers = [line for 
    page in pages[4:12] for line in page[1:-1] if any(not c in '0123456789' for c in line)][3:]

section_headers
    
core_lines = [line.strip() for page in pages[16:-1] for line in page[:-2]]

# some lines of the form 'остановить (для проверки) կանգնեցնել' mix Cyrillic and Armenian!
core_lines_fix = []
for line in core_lines:
    if isArmenian(line) and isCyrillic(line):
        core_lines_fix.append(line[:line.index(')')+1])
        core_lines_fix.append(line[line.index(')')+2:])
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# another fix, for consecutive Armenian lines. use separate loop since logic is more complex
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isArmenian(line) and isArmenian(next_line):
        if next_line.startswith(line):
            pass
        elif isArmenian(core_lines[i+2]): # only one case!
            core_lines_fix.append(line + ' ' + next_line + ' ' + core_lines[i+2])
            skip_steps = 2
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# one more fix, for consecutive Russian lines (other than headers)
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if line in section_headers:
            core_lines_fix.append(line)
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

del core_lines_fix

In [92]:
# check that all section_headers do actually appear in text
for sh in section_headers:
    print('✅' if sh in core_lines else '❌', sh)

✅ ОСНОВНЫЕ ПОНЯТИЯ
✅ основные понятия | часть 1
✅ 1. Местоимения
✅ 2. Приветствия. Прощания. Извинения. Благодарность
✅ 3. Обращения
✅ 4. Числаот1до100
✅ 5. Числа от 100
✅ 6. Числа. Порядковые числительные
✅ 7. Числа. Дроби
✅ 8. Числа. Математические действия
✅ 9. Числа. Разное
✅ 10. Самые важные глаголы - 1
✅ 11. Самые важные глаголы - 2
✅ 12. Самые важные глаголы - 3
✅ 13. Самые важные глаголы - 4
✅ 14. Цвета
✅ 15. Вопросы
✅ 16. Основные предлоги
✅ 17. Вводные и служебные слова. Наречия - 1
✅ 18. Вводные и служебные слова. Наречия - 2
✅ основные понятия | часть 2
✅ 19. Дни недели
✅ 20. Часы. Время суток
✅ 21. Месяцы. Времена года
✅ 22. Время. Разное
✅ 23. Противоположности
✅ 24. Линии и формы
✅ 25. Меры измерения
✅ 26. Ёмкости
✅ 27. Материалы
✅ 28. Металлы
✅ ЧЕЛОВЕК
✅ человек | тело человека
✅ 29. Человек. Общие понятия
✅ 30. Анатомия
✅ 31. Голова
✅ 32. Тело
✅ одежда | аксессуары
✅ 33. Верхняя одежда
✅ 34. Одежда
✅ 35. Одежда. Бельё
✅ 36. Головной убор
✅ 37. Обувь
✅ 38. Ткани. Матери

In [93]:
category = ''
subcategory = ''
group = ''

dfRows = []

for i in range(len(core_lines)):
    line = core_lines[i]
    
    if isArmenian(line):
        continue # handle all processing one line ahead of definitions
    
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if re.match(r'^\d+\. ', line):
            group = line
        if line.islower():
            subcategory = line
        if line.isupper():
            category = line
    
    if isArmenian(next_line):
        dfRows.append({
            'category': category, 'subcategory': subcategory, 'group': group,
            'russian': line, 'armenian': next_line
        })

Phrasebook = DataFrame(dfRows)

Phrasebook.armenian = Phrasebook.armenian.apply(fixDoubleLetter)

In [94]:
Phrasebook.sample(30)

,category,subcategory,group,russian,armenian
1849,ЧЕЛОВЕК,человек | чувства | разное,64. Юмор. Смех. Радость,юмор,հումոր
72,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,4. Числаот1до100,32 тридцать два,երեսուներկու
76,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,4. Числаот1до100,42 сорок два,քառասուներկու
678,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 2,23. Противоположности,про себя,մտքում
8426,РАЗНОЕ,основные глаголы | 550 слов,253. Глаголы Ж-М,заворачивать (в бумагу),փաթաթել
8014,СТРАНОВЕДЕНИЕ,страны | национальности,248. Вера. Христианство. Ислам,Священное Писание,Սուրբ Գիրք
7355,ПРИРОДА,флора,232. Зерновые,бобы,լոբազգիներ
3094,ДЕЯТЕЛЬНОСТЬ ЧЕЛОВЕКА,работа | бизнес | часть 1,104. Работа. Бизнес-процессы - 1,сделка,գործարք
6912,ПРИРОДА,фауна,215. Детёныши животных и птиц,львёнок,առյուծի ձագ
2011,ЧЕЛОВЕК,человек | чувства | разное,68. Согласие. Несогласие. Одобрение,подтвердить,հաստատել


In [95]:
Phrasebook[Phrasebook.group.str.startswith('63.')]

,category,subcategory,group,russian,armenian
1822,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,спать,քնել
1823,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,сон (состояние),քուն
1824,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,сон (сновидения),երազ
1825,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,видеть сны,երազներ տեսնել
1826,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,сонный,քնաթաթախ
1827,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,кровать,մահճակալ
1828,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,матрас,ներքնակ
1829,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,одеяло,վերմակ
1830,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,подушка,բարձ
1831,ЧЕЛОВЕК,человек | чувства | разное,63. Сон. Состояние сна,простыня,սավան


In [96]:
[word for word in Phrasebook.armenian if 
                             word[0] == word[1] and not word[0] == '.']

['թթվասեր',
 'դդում',
 'ծծակ',
 'թթվածին',
 'շշուկ',
 'կկու',
 'խխունջ',
 'դդում',
 'թթու']

In [97]:
'թթխմոր' in core_lines

False

In [98]:
fixDoubleLetter('թթու')

'թթու'

In [99]:
Phrasebook[Phrasebook.armenian==('թթվասեր')]

,category,subcategory,group,russian,armenian
1273,ЧЕЛОВЕК,питание,44. Продукты,сметана,թթվասեր
